In [1]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def get_final_image(roi_gray):
    roi = cv2.cvtColor(roi_gray, cv2.COLOR_GRAY2RGB)
    final_image = cv2.resize(roi, (224, 224))
    final_image = np.expand_dims(final_image, axis=0)
    final_image = final_image / 255.0
    return final_image

def set_label(Predictions):
    status = ""
    predict = ""
    label = np.argmax(Predictions)
    if(label == 0):
        status = "Angry"    
    if(label == 1):
        status = "Disgust"
    if(label == 2):
        status = "Fear"
    if(label == 3):
        status = "Happy"
    if(label == 4):
        status = "Neutral"
    if(label == 5):
        status = "Sad"
    if(label == 6):
        status = "Surprise"
    predict = str(Predictions[0][label])
    return (status, predict)

def draw_label_and_rectangle(img, status, predict, x, y, w, h):
    x1,y1,w1,h1 = 0,0,175,75
    cv2.rectangle(img, (x1,x1), (x1+w1,y1+h1), (0,0,0), -1)
    cv2.putText(img, status, (x1 + int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    cv2.putText(img, predict, (x1 + int(w1/10), y1+int(h1/1.3)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255))
    
def set_cap_params():
    cap.set(3,640) # set Width
    cap.set(4,480) # set Height
    cap.set(5,24) # set fps

In [4]:
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
final_model = tf.keras.models.load_model('Final_model4.h5')

cap = cv2.VideoCapture(0)
set_cap_params()

while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,     
        minSize=(100, 100)
    )

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        
    final_image = get_final_image(roi_gray)
    Predictions = final_model.predict(final_image)
    
    (status, predict) = set_label(Predictions)
    draw_label_and_rectangle(img,status, predict, x, y, w, h)
    
    cv2.imshow('video',img)

    k = cv2.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to quit
        break

cap.release()
cv2.destroyAllWindows()